In [1]:
# conda create -n quilt-env python=3.10
# conda activate quilt-env
# pip install quilt3 pydantic==2.0
# conda install ipykernel
# conda install pandas
# conda install aicsimageio


save_path = "./test_images"
metadata_save_name = save_path + "/metadata.csv"
img_save_path = save_path + "/structure_examples"

In [4]:
import quilt3 as q3
q3.Package.browse(
    "aics/hipsc_single_cell_image_dataset",
    registry="s3://allencell"
)["metadata.csv"].fetch(metadata_save_name)


Loading manifest: 100%|██████████| 484465/484465 [00:06<00:00, 71.2k/s]
100%|██████████| 1.69G/1.69G [01:22<00:00, 20.6MB/s] 


PackageEntry('file:///home/projects/cellpose/test_images/metadata.csv')

In [5]:
# 对于每种 structure_name：
# 下载 fov_path 对应的图像，提取结构通道 ChannelNumberStruct，保存为 结构名_raw.tif
# 下载 struct_seg_path 对应的分割图像，保存为 结构名_seg.tif
# 文件统一保存在 ./test_images/structure_examples/ 文件夹中
# 如果遇到问题，访问下一条记录
# 如果代码运行一直出错可能是网络问题
import pandas as pd
import quilt3 as q3
from pathlib import Path
from aicsimageio import AICSImage
from tifffile import imwrite

# 参数
MAX_TRIES = 10
output_dir = Path(img_save_path)
output_dir.mkdir(parents=True, exist_ok=True)

# 加载 metadata
df = pd.read_csv(metadata_save_name)
df_unique = df.drop_duplicates(subset="fov_path")

# quilt 包
pkg = q3.Package.browse("aics/hipsc_single_cell_image_dataset", registry="s3://allencell")

# 遍历每个结构
for structure, group in df_unique.groupby("structure_name"):
    print(f"\n▶ 正在处理结构: {structure}")
    success = False

    for _, row in group.head(MAX_TRIES).iterrows():
        try:
            # 下载并读取 fov 图像
            fov_path = row["fov_path"]
            local_fov = output_dir / "temp_fov.ome.tiff"
            if not local_fov.exists():
                pkg[fov_path].fetch(local_fov)
            img = AICSImage(local_fov)
            ch_idx = int(row["ChannelNumberStruct"])
            raw = img.get_image_data("ZYX", C=ch_idx)

            # 下载并读取 seg 图像
            seg_path = row["struct_seg_path"]
            local_seg = output_dir / "temp_seg.ome.tiff"
            if not local_seg.exists():
                pkg[seg_path].fetch(local_seg)
            seg = AICSImage(local_seg).get_image_data("ZYX")

            # 保存输出
            raw_out = output_dir / f"{structure}_raw.tif"
            seg_out = output_dir / f"{structure}_seg.tif"
            imwrite(raw_out, raw)
            imwrite(seg_out, seg)
            print(f"✅ 成功保存: {raw_out} 和 {seg_out}")

            # 删除临时文件
            local_fov.unlink(missing_ok=True)
            local_seg.unlink(missing_ok=True)

            success = True
            break
        except Exception as e:
            print(f"⚠️ 失败: {e}")
            continue

    if not success:
        print(f"❌ {structure} 的前 {MAX_TRIES} 条记录全部失败")


09-Jul-25 21:53:30 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).
/tmp/ipykernel_103616/3351000164.py:19: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_save_name)
Loading manifest: 100%|██████████| 484465/484465 [00:06<00:00, 74.1k/s]



▶ 正在处理结构: AAVS1


100%|██████████| 423M/423M [00:23<00:00, 18.1MB/s]  
100%|██████████| 615k/615k [00:04<00:00, 146kB/s] 
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


✅ 成功保存: test_images/structure_examples/AAVS1_raw.tif 和 test_images/structure_examples/AAVS1_seg.tif

▶ 正在处理结构: ACTB


100%|██████████| 441M/441M [00:20<00:00, 21.8MB/s]  
100%|██████████| 469k/469k [00:04<00:00, 112kB/s] 
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


...

✅ 成功保存: test_images/structure_examples/TJP1_raw.tif 和 test_images/structure_examples/TJP1_seg.tif

▶ 正在处理结构: TOMM20


100%|██████████| 372M/372M [00:22<00:00, 16.8MB/s]  
100%|██████████| 599k/599k [00:07<00:00, 83.5kB/s]
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


✅ 成功保存: test_images/structure_examples/TOMM20_raw.tif 和 test_images/structure_examples/TOMM20_seg.tif

▶ 正在处理结构: TUBA1B


100%|██████████| 300M/300M [00:33<00:00, 9.01MB/s]  
100%|██████████| 1.21M/1.21M [00:04<00:00, 279kB/s]
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/root/anaconda3/envs/quilt-env/lib/python3.10/site-packages/pydantic/main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


✅ 成功保存: test_images/structure_examples/TUBA1B_raw.tif 和 test_images/structure_examples/TUBA1B_seg.tif


In [2]:
# 在下载一个间期的Lamin B1
import pandas as pd
import quilt3 as q3
from pathlib import Path
from aicsimageio import AICSImage
from tifffile import imwrite

output_dir = Path(img_save_path)
output_dir.mkdir(parents=True, exist_ok=True)

# 加载 metadata
df = pd.read_csv(metadata_save_name)
df_unique = df.drop_duplicates(subset="fov_path")

# quilt 包
pkg = q3.Package.browse("aics/hipsc_single_cell_image_dataset", registry="s3://allencell")
# 下载并读取 fov 图像
fov_path = "fov_path/c6e4652401c17c7261129771e35fd5c7dfea510fc5f2e11fc53d55c0ca9aa6a2_3500000876_100X_20170508_3-Scene-07-P27-E06.ome.tiff"
local_fov = output_dir / "temp_fov.ome.tiff"
if not local_fov.exists():
    pkg[fov_path].fetch(local_fov)
img = AICSImage(local_fov)
ch_idx = 3
raw = img.get_image_data("ZYX", C=ch_idx)

# 下载并读取 seg 图像
seg_path = "struct_seg_path/48d8a51baa42ec1026965c4caef0e2fb5fd0cf6293be816826315d34fba949c3_3500000876_100X_20170508_3-Scene-07-P27-E06_struct_segmentation.tiff"
local_seg = output_dir / "temp_seg.ome.tiff"
if not local_seg.exists():
    pkg[seg_path].fetch(local_seg)
seg = AICSImage(local_seg).get_image_data("ZYX")

# 保存输出
raw_out = output_dir / "LMNB1_M0_raw.tif"
seg_out = output_dir / "LMNB1_M0_seg.tif"
imwrite(raw_out, raw)
imwrite(seg_out, seg)
print(f"✅ 成功保存: {raw_out} 和 {seg_out}")

# 删除临时文件
local_fov.unlink(missing_ok=True)
local_seg.unlink(missing_ok=True)


09-Jul-25 23:09:34 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).
/tmp/ipykernel_264666/845478045.py:12: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_save_name)
Loading manifest: 100%|██████████| 484465/484465 [00:06<00:00, 74.7k/s]
09-Jul-25 23:10:01 - tifffile.tifffile - WARNING  - <tifffile.TiffPage 0 @8> <tifffile.TiffTag 270 @70> invalid value offset 411711337
09-Jul-25 23:10:01 - tifffile.tifffile - WARNING  - <tifffile.TiffPage 0 @8> <tifffile.TiffTag 270 @70> invalid value offset 411711337
09-Jul-25 23:10:01 - tifffile.tifffile - WARNING  - <tifffile.TiffPage 0 @8> <tifffile.TiffTag 270 @70> invalid value offset 411711337
09-Jul-25 23:10:01 - tifffile.tifffile - WARNING  - <tifffile.TiffPage 0 @8> <tifffile.TiffTag 270 @70> invalid value offset 411711337
09-Jul-25 23:10:01 - tifffile.tifffile - WARNING  - <tifffile.TiffPage 0 @8> <t

TiffFileError: corrupted strip cannot be reshaped from (912,) to (1, 1, 924, 1)